In [ ]:
%%capture
!pip install yfinance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

In [ ]:
#ALGORITHM OUTLINE
# (1) Find current_price for stock of interest (use stocks most recent close price)
# (2) Check if .93*current_price \in [low,high] for stock of interest for the last week (last five days)

In [ ]:
# algorithm for single date
# dat = yf.Ticker("^TASI.SR")
# history_for_lastweek_5d = dat.history(period='5d')
# low = history_for_lastweek_5d["Low"].min()
# high = history_for_lastweek_5d["High"].max()
# current_price = history_for_lastweek_5d["Close"][0]
# price_of_interest = .93*current_price
# price_of_interest_within_last_week = low <= price_of_interest and price_of_interest <= high

<ipython-input-70-2770be6dda2d>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = history_for_lastweek_5d["Close"][0]


In [112]:
# algorithm for every date between start date and end date
ticker = "^TASI.SR"
data = yf.download(ticker, start="2010-01-01", end="2025-01-24")
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)

data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)

data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data = data.drop(["Low_Last_5d", "High_Last_5d", "seven_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)
data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)
data = data.drop(["close_up_diff", "close_down_diff"], axis=1)
data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
data.tail(3)

[*********************100%***********************]  1 of 1 completed


Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,closeaboveup1%,closebelowdown1%,aboveup1%_or_belowdown1%
Ticker,^TASI.SR,^TASI.SR,^TASI.SR,,,,,,,
Date,,,,,,,,,,
2025-01-21,12369.629883,12393.370117,293385400,False,53,11898.214258,11662.606055,True,False,True
2025-01-22,12362.389648,12376.379883,346099500,False,53,11898.214258,11662.606055,True,False,True
2025-01-23,12354.040039,12368.250000,252296200,False,53,11898.214258,11662.606055,True,False,True


In [113]:
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
data.head(3)

Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,closeaboveup1%,closebelowdown1%,aboveup1%_or_belowdown1%
Ticker,^TASI.SR,^TASI.SR,^TASI.SR,,,,,,,
Date,,,,,,,,,,
2010-06-21,6450.680176,6413.100098,156146200,False,1,6403.430083,6276.629487,True,False,True
2010-06-29,6135.859863,6302.580078,131370700,False,1,6403.430083,6276.629487,False,True,True
2010-06-30,6093.759766,6135.859863,106756200,False,1,6403.430083,6276.629487,False,True,True


In [114]:
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)

In [117]:
print(data["closeaboveup1%"].sum())
print(data["closebelowdown1%"].sum())

8
18


In [103]:
'''
Explanation:
If we had purchased "^TASI.SR" evrytime the stock had decreased at least seven percent in the last five trade days, and sold the stock and exited everytime a one percent
loss was realized or exited evertime a one percent gain was realized between startdate = "2010-01-01" and enddate = "2025-01-24"we would have realized profit on eight trades
and realized loss on eighteen trades.
'''

(27, 10)